In [ ]:
# Testing Cell
from aviary.subsystems.atmosphere.atmosphere import AtmosphereComp
from aviary.subsystems.atmosphere.data.StandardAtm1976 import atm_data
from aviary.subsystems.atmosphere.data.MIL_SPEC_210A_Cold import atm_data
from aviary.subsystems.atmosphere.data.MIL_SPEC_210A_Hot import atm_data
from aviary.subsystems.atmosphere.data.MIL_SPEC_210A_Polar import atm_data
from aviary.subsystems.atmosphere.data.MIL_SPEC_210A_Tropical import atm_data
import openmdao.api as om

from aviary.subsystems.atmosphere.utils import build_akima_coefs

# Atmosphere Subsystem
Aviary has several built-in atmosphere models that support the Aerodynamics calculations, by providing temperature, pressure, speed of sound, and viscosity based on an input altitude. There are two different atmosphere models implemented in Aviary, one coming from the 1976 standard atmosphere tables and the other coming from the publically released MIL-SPEC-210A tables. The standard 1976 tables are the default option and represent the typical conditions experienced at a given altitude. However, it is important to note that "the atmosphere has never, and will never be, standard". All of the atmosphere models are approximations. The MIL-SPEC-210A tables represent extreme atmospheres in tropical, arctic (polar winter), extreme hot, and extreme cold climates. 

To select the atmosphere type, you can use `options.set_val(Settings.ATMOSPHERE_MODEL, val=AtmosphereModel.STANDARD)`


# Comparison between Different Models
A few graphs comparing the different models have been included so the user can visually understand the different atmosphere models.
![images/Atmosphere_Profile_Temp.png](images/Atmosphere_Profile_Temp.png)
![images/Atmosphere_Profile_Pressure.png](images/Atmosphere_Profile_Pressure.png)
![images/Atmosphere_Profile_Density.png](images/Atmosphere_Profile_Density.png)


# How the Atmosphere Model Works
Examining `atmosphereComp.py` will show the code on how the atmosphere component is implemented. Generally speaking, akima splines that represent the raw data are loaded into the model during initialization. The akima splines are used to enhances computational speed. During every subsequent call to the atmosphere mode, an input altitude is translated via the akima splines into a temperature, pressure, density and then additional calculations are performed to determine speed of sound, and dynamic viscosity. The raw data and the akima splines of that data are contained in individual files (`StandardAtm1976.py`, `MIL_SPEC_210A_cold.py`, `MIL_SPEC_210A_hot.py`, `MIL_SPEC_210A_polar.py`, `MIL_SPEC_210A_tropical.py`). 

# User options reference for AtmosphereComp and AtmosphereGroup
There are some critical options you can pass to atmosphere that you should know about. These options will allow you to change which source data you use, swap between geocentric or geodetic altitude inputs, and add a temperature offset. Changing the source data swaps the akima splines used to calculate temperature, pressure, and density as a function of altitude. 

All the options and their defaults and functions are shown below.

In [ ]:
om.show_options_table('aviary.subsystems.atmosphere.atmosphere.AtmosphereComp')

## Temperature Offset Options
Adding a temperature offset can enable simulation with especially hot or cold days which is useful in acoustic certification of aircraft.  When inputting a temperature offset as an option, the temperature throughout the entire altitude regime is shifted up or down by this ammount. The pressure is assumed to remain constant. This is a simplification we must adhere to because we do not have any additional pressure data or assumptions to build upon. Density is then recalculated as an offset from the source data. This is important because that source data may differ from the ideal gas equations used to calculate density. This is the case for the MIL-SPEC-210A data.

# US 1976 Atmosphere Notes
The US 1976 atmospheres source information and notes are as follows:

In [ ]:
from aviary.subsystems.atmosphere.data.StandardAtm1976 import DATA_ORIGIN_NOTE

print(DATA_ORIGIN_NOTE)

# MIL-SPEC-210A Atmosphere Notes
The MIL-SPEC-210A atmospheres (including hot/cold/tropical/polar) source information and notes are as follows:

In [ ]:
from aviary.subsystems.atmosphere.data.MIL_SPEC_210A_Tropical import DATA_ORIGIN_NOTE

print(DATA_ORIGIN_NOTE)

# Adding your own Atmosphere Model
If you desire to add your own atmosphere model you can do this one of two ways. Either create a new `AtmosphereComp.py` from scratch, or by importing a new set of raw data and using the current `atmosphereComp.py` to read it in and process it. This will include first saving your new raw data set in a new file and then creating a new set of akima splines for that data and also saving them in the same file. After that, `utils/build_akima_coefs.py` can read in the akima's and run as normal. This walkthrough will help you perform the latter option. Start by creating a new file to hold your raw atmosphere model and the eventual akima splines. 

## Create a new Atmosphere file
You will need to make a new file to hold the raw data of your atmosphere model and the akima splines. This file should be titled after your atmosphere model, i.e. `StandardAtm1976` or `MIL_SPEC_210A_Cold`. 
Save that file inside of the `aviary/subsystems/atmosphere/data` folder.
You will need to make a new `_raw_data` dictionary inside your new file. Look at the example `_raw_data` contained in `data/StandardAtm1976` for reference. 

## Converting Raw data Proper Units
From here on our we need to be especially mindful of units. We are going to use a utility function `build_akima_coefs` to help create the akima coefficients. The helper function is located in `utils/build_akima_coefs.py`. 
We only need to build the akima coefficients once and we do it so that the atmosphere model is very fast. The atmosphere model will need the following information: (Altitude, Temperature, Pressure, Density). If you don't have one of those values that's ok, typically given Altitude, Temperature, and some assumption of the pressure at sea level, all other values can be calculated based on ideal gas assumptions.   

Your units inside of `_raw_data` should be one of the following: 
English: [altitude (ft, ), temperature (degF), pressure (inHg at 60degF), density (lbm/ft**3)]
Altitude can be in either in geodetic or deocentric
Pressure will only translate properly if given in inHg at 60degF. However, many data sets use inHg at 30degF. 

SI Units: [altitude (m), temperature (degK), pressure (millibar), density (kg/m^3)]
Altitude can be in either in geodetic or deocentric

If your units do not match either English or SI as designated above, you will need to modify the `build_akima_coefs` function to properly translate your raw data into SI units. (All calculations inside of atmosphere.py are happening in SI units with SI akima splines)

## Processing new Akima Splines
Next, open up `utils/build_akima_coefs.py`, you will notice that at the bottom of the file there is an call to `if __name__ == '__main__':`. Change the import location to point to your new `_raw_data` file and change the units to be either SI or English, depending on your raw data. Then execute `python atmosphereComp.py`, this will call the `build_akima_coefs` to build the akima splines using your raw data. Copy those splines into your new atmosphere file you made a few steps earlier and save the file. 

# Add the new data to Enums
You also need to add your new atmosphere model to the Enums so that it shows up automatically for ease of use:
Head over to `aviary.variable_info.enums.py` and modify the `class AtmosphereModel(Enum):` to add your new atmosphere model name.

## Import the Processed Data
Next we will need to import these akima splines into `AtmosphereComp()` i.e.

`from aviary.subsystems.atmosphere.data.NEW_DATA_FILE import atm_data as new_data`

Lastly in `Setup()` section of `AtmosphereComp()` there is a section of code that loads the atmosphere model based on the user specified option. Add a new line there to properly load youre new file:

`elif self.options[Settings.ATMOSPHERE_MODEL] is AtmosphereModel.'new_name': self.source_data = new_data`

## Test Your Model
Now that you have your model loaded and ready to go, use the manual test at the bottom of `utils/build_akima_coefs.py` to test inputting a few altitude values into your model and inspect the resulting output of temperature, pressure, density, speed of sound, and dynamic viscosity. 